In [ ]:
!pip install datasets transformers torch peft --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Check if bitsandbytes is available
try:
    import bitsandbytes as bnb
    from transformers import BitsAndBytesConfig
    use_4bit = True
    print("Using 4-bit quantization with bitsandbytes")
except ImportError:
    use_4bit = False
    print("bitsandbytes not found. Falling back to 16-bit training.")

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Load the tokenizer
model_name = "bert-base-uncased"  # You can change this to any other suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load the model
if use_4bit:
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, quantization_config=bnb_config)
else:
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define LoRA Config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

# Get the PEFT model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    learning_rate=2e-5,
    fp16=True,  # Enable mixed precision training
)

# Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

# Perform evaluation before training
print("Evaluating before training...")
pre_train_metrics = trainer.evaluate()
print(f"Pre-training metrics: {pre_train_metrics}")

# Start training
print("Starting training...")
trainer.train()

# Perform evaluation after training
print("Evaluating after training...")
post_train_metrics = trainer.evaluate()
print(f"Post-training metrics: {post_train_metrics}")

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Print improvement
print(f"Accuracy improvement: {post_train_metrics['eval_accuracy'] - pre_train_metrics['eval_accuracy']}")

bitsandbytes not found. Falling back to 16-bit training.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

trainable params: 591,362 || all params: 110,075,140 || trainable%: 0.5372
Evaluating before training...


Pre-training metrics: {'eval_loss': 0.7862803339958191, 'eval_accuracy': 0.49996, 'eval_runtime': 57.7526, 'eval_samples_per_second': 432.881, 'eval_steps_per_second': 27.064}
Starting training...


Step,Training Loss,Validation Loss,Accuracy
500,0.682900,0.622682,0.668360
1000,0.371100,0.322510,0.870800
1500,0.265700,0.306460,0.881960
2000,0.297100,0.291829,0.889600
2500,0.301100,0.316427,0.883520
3000,0.285900,0.279742,0.898880
3500,0.277100,0.283108,0.897600
4000,0.204300,0.270664,0.905880
4500,0.391400,0.284527,0.906320
5000,0.386900,0.260939,0.908120


Evaluating after training...


Post-training metrics: {'eval_loss': 0.2581176459789276, 'eval_accuracy': 0.91296, 'eval_runtime': 56.1386, 'eval_samples_per_second': 445.326, 'eval_steps_per_second': 27.842, 'epoch': 3.0}
Accuracy improvement: 0.413
